# NIRCAM - Finding and validating the location of object specta for the WFSS mode
There are no NIRCAM grism simulations available here: http://archive.stsci.edu/jwst/simulations/nircam.html
so the following example uses images supplied by the team. Contact Bryan Hilbert.

This notebook will walk you through extracting grism spectra using a catalog of sources. 

## Make sure that you have set the JWST_NOTEBOOK_DATA environment variable in the terminal from which you started Jupyter Notebook.

The data will be read from that directory, and the pipeline should write to the current working directory, avoiding clobbers.
If you would like to use your own data just substitute the locations below.

In [ ]:
# plotting, the inline must come before the matplotlib import
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.patches as patches

params = {'legend.fontsize': 6,
          'figure.figsize': (8, 8),
          'figure.dpi': 150,
         'axes.labelsize': 6,
         'axes.titlesize': 6,
         'xtick.labelsize':6,
         'ytick.labelsize':6}
plt.rcParams.update(params)


# python general
import os
import numpy as np

# astropy modules
import astropy
from astropy.io import fits
from astropy.table import QTable
from astropy.wcs.utils import skycoord_to_pixel
import photutils

# jwst 
import jwst
from jwst.source_catalog import source_catalog_step
from jwst.datamodels import image, DrizProductModel, WavelengthrangeModel
from jwst import assign_wcs
from jwst.assign_wcs import assign_wcs_step

In [ ]:
print("Using jwst pipeline version: {}\nastropy version: {}\n photutils version: {}".format(jwst.__version__, astropy.__version__, photutils.__version__ ))

## Example model data from the science team

    NIRCam WFSS datasets for pipeline testing:

    12 pointings (3 dither positions, with 4 subpixel dithers),
    for each of row and column dispersed spectra. Associated direct
    images and out-of-field observations, along with
    all of the associated shortwave channel data, which is
    all imaging mode.


    Filenames are (e.g.) V54321001002P0000000001101_A1_F150W_uncal.fits
    for imaging files, and
    V54321001003P0000000001116_A5_F444W_FinalDispersedRamp_XXX_uncal.fits
    for dispersed files, where XXX can be 'row' or 'column'.



    Cheat sheet:
    File numbers below are the 4 digits immediately before the first
    underscore in the filename.


    File number       Description


    **************************************
    ********Observation 2*****************

    1101 - 1109       LW:12 dithered grism observations, row dispersed, and
    110a - 110c       complimentary SW imaging data

    110d              LW: Direct image, SW: complimentary imaging data

    110e and 110f     out-of-field observations

    110g to 110r      LW:12 dithered grism observations, column dispersed,
                      and SW: complimentary imaging data

    110s              LW: Direct image, SW: complimentary imaging data

    110t and 110u     out-of-field observations
    
    **************************************
    *********Observation 3****************
    110v - 110z       LW:12 dithered grism observations, row dispersed, and
    1110 - 1116       SW:complimentary imaging data

    1117              LW: Direct image, SW: complimentary imaging data

    1118 - 1119       out-of-field observations

    111a - 111l       LW:12 dithered grism observations, column dispersed,
                      and SW: complimentary imaging data

    111m              LW: Direct image, SW: complimentary imaging data

    111n - 111o       out-of-field observations




### I've selected the following datasets to use as examples in this notebook. The other datasets can also be validated in the same manner.

#### Row dispersed data
    V54321001002P000000000110d_A5_F444W_rate.fits
    V54321001002P0000000001101_A5_F444W_FinalDispersedRamp_row_rate.fits

#### Column dispersed data
    V54321001002P000000000110g_A5_F444W_FinalDispersedRamp_column_rate.fits
    V54321001002P000000000110s_A5_F444W_rate.fits


In [ ]:
notebook_dir = os.environ['JWST_NOTEBOOK_DATA']
nircam_data = notebook_dir + 'nircam/'

grismr_file = nircam_data + 'V54321001002P0000000001101_A5_F444W_FinalDispersedRamp_row_rate.fits'  # this is a row dispersed grism image
directr_file = nircam_data + 'V54321001002P000000000110d_A5_F444W_rate.fits'  # this is a filtered image
grismc_file = nircam_data + 'V54321001002P000000000110g_A5_F444W_FinalDispersedRamp_column_rate'  # this is a column dispersed grism image
directc_file = nircam_data + 'V54321001002P000000000110s_A5_F444W_rate.fits'  # this is a filtered image

#### let's look at what the row dispersed images contain

You should already be able to tell the grism image was taken at a different pointing that the direct image

In [ ]:
dispim=fits.open(grismr_file)
dirim=fits.open(directr_file)
ys,xs=dirim[1].data.shape
fig = plt.figure(figsize=(8,8), dpi=150)
ax = fig.add_subplot(1, 2, 1)
ax.set_title(directr_file.split("/")[-1], fontsize=8)
ax.imshow(dirim[1].data, origin='lower', extent=[0,xs,0,ys], vmin=-3, vmax=3)
ax2 = fig.add_subplot(1, 2, 2)
ax2.set_title(grismr_file.split("/")[-1], fontsize=8)
ax2.imshow(dispim[1].data, origin='lower', extent=[0,xs,0,ys], vmin=-3, vmax=3)
fig.tight_layout()
fig.savefig('nircam_image_comparison.jpg')
dispim.close()
dirim.close()

#### You should be able to tell from the above images that the grism image has been taken at a different pointing from the direct image

## We are going to create a source catalog for our direct image
If you've already worked through the notebook in general/Create-Source-Catalog notebook the following cells will look familiar

#### Create the source catalog step object

In [ ]:
sc=source_catalog_step.SourceCatalogStep(save_results=True)

In [ ]:
print(sc.spec)  # display the defaults

### Open our image as a datamodel and find the sources

In [ ]:
dpm=DrizProductModel(directr_file)

#### check out some quick information about the image we are using

In [ ]:
dpm.meta.subarray.name, dpm.meta.instrument.name, dpm.meta.instrument.detector, dpm.meta.instrument.channel, dpm.meta.instrument.filter, dpm.meta.instrument.pupil, dpm.meta.instrument.module, dpm.meta.exposure.type

In [ ]:
# Add this if your direct image hasn't been through resample
dpm.meta.resample.product_exposure_time = dpm.meta.exposure.exposure_time

In [ ]:
# source catalog is using get_fits_wcs() and that barfs on the DrizProdModel if validation fails on VAR_POISSON and VAR_RNOISE
# those don't exist are set to unloaded arrays in the model, so here  we'll set them.

dpm.var_poisson = 0
dpm.var_rnoise = 0

In [ ]:
fits.info(directr_file)

In [ ]:
sc.process(dpm)

#### The above results should print out the name of the catalog that was created, my first run showed 2718 with the default parameters, we'll change that to get better detections below

#### If you see a TypeError, it's most likely because your image hasn't been sent through resample so it's missing a WHT extension and some meta information. We can add one ourselves and continue. If not, you can skip the following cells.

In [ ]:
# execute to add WHT extension to dataset
dirim = fits.open(directr_file, mode='update')

WHT = False
for extname in dirim:
    if 'WHT' in extname.name:
        WHT = True
if not WHT:
    wht_hdu = fits.ImageHDU()
    wht_hdu.data = np.ones((2048, 2048), dtype=np.float32)
    wht_hdu.data
    wht_hdu.header['EXTNAME'] = 'WHT'
    dirim.append(wht_hdu)
else:
    print("WHT extension already exists, no update")
    
dirim.close()

In [ ]:
fits.info(directr_file)

#### Edit the defaults for source finding and run again to get a better detection sample


In [ ]:
sc.npixels=50
sc.snr_threshold=30
sc.process(dpm)

In [ ]:
skycoord_cat='step_SourceCatalogStep_cat.ecsv'  # this should be the name in the output listing above
#skycoord_cat='V54321001002P000000000110d_A5_F444W_rate_cat.ecsv'
catalog=QTable.read(skycoord_cat,  format='ascii.ecsv')
catalog

#### the `source_catalog` step returns the xcentroid and ycentroid in pixel values, let's plot those over our image to see how the detections look

In [ ]:
dirim = fits.getdata(directr_file, ext=1)
xs,ys=dirim.shape
fig = plt.figure(figsize=(4,4), dpi=150)
ax = fig.add_subplot(1, 1, 1)
ax.ticklabel_format(useOffset=False)
ax.ticklabel_format(useOffset=False)
ax.set_title(directr_file.split("/")[-1], fontsize=8)
ax.imshow(dirim, origin='lower', extent=[0,xs,0,ys], vmin=-10, vmax=10)

# rectangle patches are xmin, ymin, width, height
plist1=[]
for obj in catalog:
    plist1.append(patches.Circle((obj['xcentroid'].value, obj['ycentroid'].value),4, color='red'))
    ax.text(obj['xcentroid'].value, obj['ycentroid'].value, obj['id'], size=5)

for p in plist1:
    ax.add_patch(p)
    
ax.imshow(dirim, origin='lower', extent=[0,xs,0,ys], vmin=-10, vmax=10)
print("Saved image to nircam_wfss_catalog_centers.jpg")
fig.savefig('nircam_wfss_catalog_centers.jpg')

### source_catalog returns the minimal bounding boxes around each object in units of degrees:
* sky_bbox_ll
* sky_bbox_ul
* sky_bbox_lr
* sky_bbox_ur

#### Let's translate those corners so that we can overplot our bounding boxes for each object
**Caveat, as of the writing of this notebook, source_catalog hasn't been fully updated to use GWCS objects, so we are going to do the radec --> pixel translation using the fits wcs in the model 

In [ ]:
wcs = dpm.get_fits_wcs(hdu_name='SCI')

In [ ]:
wcs.celestial

### Use the astropy.wcs object to convert the bounding box corners for all objects to pixel locations

In [ ]:
det_bbox_ll_x, det_bbox_ll_y  = skycoord_to_pixel(catalog['sky_bbox_ll'], wcs, origin=0)
det_bbox_lr_x, det_bbox_lr_y  = skycoord_to_pixel(catalog['sky_bbox_lr'], wcs, origin=0)
det_bbox_ul_x, det_bbox_ul_y  = skycoord_to_pixel(catalog['sky_bbox_ul'], wcs, origin=0)
det_bbox_ur_x, det_bbox_ur_y  = skycoord_to_pixel(catalog['sky_bbox_ur'], wcs, origin=0)

In [ ]:
catalog[18]

### Plot up the minimal bounding boxes that were computed with the catalog for the direct image. The cross-dispersion size of the grism extraction boxes will be taken directly from these values. The `source_catalog` step is currently using the fits wcs to calculate the sky poisitions. The pipeline uses the GWCS object though.

In [ ]:
# display and save a jpeg copy of the results 
dispim=fits.getdata(grismr_file, ext=1)
dirim=fits.getdata(directr_file, ext=1)
ys,xs=dirim.data.shape
fig2 = plt.figure(figsize=(8,8), dpi=150)
ax1 = fig2.add_subplot(1, 2, 1)
ax1.ticklabel_format(useOffset=False)
ax1.set_title(directr_file.split("/")[-1], fontsize=8)
ax2 = fig2.add_subplot(1, 2, 2)
ax2.set_title(grismr_file.split("/")[-1], fontsize=8)

# rectangle patches are xmin, ymin, width, height
plist1=[]
for xmin,ymin,xmax,ymax in zip(det_bbox_ll_x, det_bbox_ll_y, det_bbox_ur_x,det_bbox_ur_y):
    plist1.append(patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, fill=False, color='red'))

for p in plist1:
    ax1.add_patch(p)
    
ax1.imshow(dirim, origin='lower', extent=[0,xs,0,ys], vmin=-10, vmax=10)
ax2.imshow(dispim, origin='lower', extent=[0,xs,0,ys], vmin=-5, vmax=5)
saveto = directr_file.split("/")[-1].split(".fits")[0]+"_boxes.jpg"
fig2.tight_layout()
fig2.savefig(saveto, dpi=120, bbox_inches='tight')
print("Saved jpeg file to: {}".format(saveto))

### Now we need to translate the minimum bounding boxes to the grism image
We'll do this by going through part of the GWCS object for the dispersed image

**Look at the nircam/Using-the-NIRCAM-WFSS-WCS-Object  notebook for explicit detail on the GWCS object, transforms and models for WFSS**

*Since the grism image is using the GWCS object at all stages, here we could  create the GWCS object for the direct image and use that to translate the bounding box corners, and object centers, to RA,DEC locations on the sky.
We would then use the GWCS object in the grism image to translate them back to detector pixel locations. The biggest difference here between the fits WCS and the GWCS is the trip through the distorion correction.* So there's errors in the location whether you do this through fits wcs or gwcs because the catalog is only using fits wcs right now.

In [ ]:
# Lets read the file into a datamodel and display some information
grism_image = image.ImageModel(grismr_file)
grism_image.meta.instrument.name, grism_image.meta.instrument.filter, grism_image.meta.instrument.pupil, grism_image.meta.instrument.detector,grism_image.meta.exposure.type

### Take a look at what is in the wcsinfo structure

In [ ]:
gim=grism_image
gim.meta.wcsinfo.crval1, gim.meta.wcsinfo.crval2, gim.meta.wcsinfo.crpix1, gim.meta.wcsinfo.crpix2, gim.meta.wcsinfo.crpix3

In [ ]:
gim.meta.wcsinfo.wcsaxes, gim.meta.wcsinfo.ra_ref, gim.meta.wcsinfo.dec_ref, gim.meta.wcsinfo.v2_ref, gim.meta.wcsinfo.v3_ref

In [ ]:
gim.meta.wcsinfo.roll_ref, gim.meta.wcsinfo.cdelt1, gim.meta.wcsinfo.cdelt2, gim.meta.wcsinfo.cdelt3

In [ ]:
gim.meta.wcsinfo.pc1_1, gim.meta.wcsinfo.pc1_2, gim.meta.wcsinfo.pc2_1, gim.meta.wcsinfo.pc2_2

In [ ]:
junk=fits.open(grismr_file, mode='update')
junk[0].header['exp_type'] = 'NRC_WFSS'
junk.close()
fits.getval(grismr_file,'EXP_TYPE')

### The original file still had NRC_GRISM instead of NRC_WFSS for exposure.type, update if necessary

In [ ]:
wcs_step = assign_wcs.AssignWcsStep()

In [ ]:
grism_with_wcs = wcs_step(grism_image)

In [ ]:
grism_with_wcs.meta.wcs.available_frames  # this is where the WCS lives, available frames gives you the transforms that are available

#### What does the GWCS wcs information look like?

In [ ]:
grism_with_wcs.meta.wcs

#### This is what the FITS wcs information looks like for the grism image:

In [ ]:
grism_fits_wcs = grism_image.get_fits_wcs()
grism_fits_wcs

In [ ]:
# compare with the wcsinfo in the datamodel
grism_with_wcs.meta.wcsinfo.crval1, grism_with_wcs.meta.wcsinfo.crval2, grism_with_wcs.meta.wcsinfo.crpix1, grism_with_wcs.meta.wcsinfo.crpix2

### Assuming that the grism image we are working with has the correct meta information about it's pointing, we can use the RA,DEC coordinates of the corners of the boxes to translate to a respective "direct image" that might have been taken at the same location as the grism image pointing.

In [ ]:
catalog[18]

In [ ]:
# Translated locations to effective grism image "direct image" using the FITS wcs information
gdet_bbox_ll_x, gdet_bbox_ll_y  = skycoord_to_pixel(catalog['sky_bbox_ll'], grism_fits_wcs, origin=0)
gdet_bbox_lr_x, gdet_bbox_lr_y  = skycoord_to_pixel(catalog['sky_bbox_lr'], grism_fits_wcs, origin=0)
gdet_bbox_ul_x, gdet_bbox_ul_y  = skycoord_to_pixel(catalog['sky_bbox_ul'], grism_fits_wcs, origin=0)
gdet_bbox_ur_x, gdet_bbox_ur_y  = skycoord_to_pixel(catalog['sky_bbox_ur'], grism_fits_wcs, origin=0)
xcenter,ycenter = skycoord_to_pixel(catalog['sky_centroid'], grism_fits_wcs, origin=0)

### Visual display of what the translation from object locations on our direct image to object locations on our grism "direct image"
The boxes in the grism image are at the location of the obect in the catalog, but at the pointing of the grism image so they should show the dither offset
The blue dots on the grism image mark the object center, just as translated by the fits_wcs, not the full trace size. See https://jwst-docs.stsci.edu/display/JTI/NIRCam+Grisms figure 3 for the example of how the source position relates to the trace location.

In [ ]:
# display and save a jpeg copy of the results 
dispim=fits.getdata(grismr_file, ext=1)
dirim=fits.getdata(directr_file, ext=1)
ys,xs=dirim.data.shape
fig2 = plt.figure(figsize=(8,8), dpi=150)
ax1 = fig2.add_subplot(1, 2, 1)
ax1.ticklabel_format(useOffset=False)
ax1.set_title(directr_file.split("/")[-1], fontsize=8)
ax2 = fig2.add_subplot(1, 2, 2)
ax2.set_title(grismr_file.split("/")[-1], fontsize=8)

# rectangle patches are xmin, ymin, width, height
plist1=[]
for xmin,ymin,xmax,ymax,xc,yc in zip(gdet_bbox_ll_x, gdet_bbox_ll_y, gdet_bbox_ur_x, gdet_bbox_ur_y,xcenter,ycenter):
    plist1.append(patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, fill=False, color='red'))
    plist1.append(patches.Circle((xc, yc),4, color='blue'))

for p in plist1:
    ax2.add_patch(p)
    
plist1=[]
for xmin,ymin,xmax,ymax in zip(det_bbox_ll_x, det_bbox_ll_y, det_bbox_ur_x,det_bbox_ur_y):
    plist1.append(patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, fill=False, color='red'))
    plist1.append(patches.Circle((xmin, ymin),4, color='blue'))

for p in plist1:
    ax1.add_patch(p)
    
    
ax1.imshow(dirim, origin='lower', extent=[0,xs,0,ys], vmin=-10, vmax=10)
ax2.imshow(dispim, origin='lower', extent=[0,xs,0,ys], vmin=-5, vmax=5)
saveto = grismr_file.split("/")[-1].split(".fits")[0]+"_F444w.jpg"
fig2.tight_layout()
fig2.savefig(saveto, dpi=120, bbox_inches='tight')
print("Saved jpeg file to: {}".format(saveto))

### Now that we have the pixel locations from our fake direct image, we'll use  the transform from 'detector' to 'grism_detector' to translate the bounding box corner locations to the grism image itself

#### We are also going to need to use the `wavelengthrange` reference file
This tells us what the min and max wavelengths are for each spectral order and we'll use it to set the size of our extraction box in the dispersed image

#### Ask CRDS for the name of the wavelengthrange reference file that goes with this image
I'm doing this the long way as an example of how to do it by hand for any file.

How did I know how to populate the header information? Look at the rmaps in CRDS to see what its selecting against for your reference file. For NIRCAM wavelengthrange look here: https://jwst-crds.stsci.edu/browse/jwst_nircam_wavelengthrange_0008.rmap

In [ ]:
import crds
from crds import client
jwst_context = client.get_default_context('jwst')
header = {"meta.instrument.name": grism_with_wcs.meta.instrument.name,
          "meta.instrument.module": grism_with_wcs.meta.instrument.module,
          "meta.exposure.type": grism_with_wcs.meta.exposure.type,
          "meta.observation.date": grism_with_wcs.meta.observation.date,
          "meta.observation.time": grism_with_wcs.meta.observation.time,
         }
wavelengthrange = client.get_best_references(jwst_context, header, ["wavelengthrange"])
wavelengthrange

#### If you wanted to get a list of all the reference files that were needed by a particular jwst pipeline step, you could do this:


In [ ]:
from jwst.extract_2d.extract_2d_step import Extract2dStep
e2d = Extract2dStep()
ref_types = e2d.reference_file_types
ref_types

In [ ]:
reference_file_names = {}
for name in ref_types:
    reffile = e2d.get_reference_file(grism_with_wcs, name)
    reference_file_names[name] = reffile if reffile else ""
reference_file_names

**No file should have been returned for wavecorr because it's not currently used in this mode**

#### The cell below opens the Wavelengthrange refrence selected for use with our image

In [ ]:
with WavelengthrangeModel(reference_file_names['wavelengthrange']) as f:
        wavelengthrange = f.wavelengthrange
        waverange_selector = f.waverange_selector
        order = f.order
f.instance  # show everything in the file

#### The wavelength ranges are contained in the `wavelengthrange` data member, which is a list of (order, filter, wave min, wave max)
The range can be selected using just the wavelengthrange member, in the most recent reference file, the `waverange_selector` and `order` members represent lists of the available for easy reference


In [ ]:
print("wavelengthrange: {}\n\nwaverange_selector: {}\n\norder: {}".format(wavelengthrange, waverange_selector, order))

#### Find the wavelength ranges to use with the filter we have for the order we specify, make sure they are correct

In [ ]:
order = 1
range_select = [(x[2], x[3]) for x in wavelengthrange if (x[0] == order and x[1] == grism_image.meta.instrument.filter)]
lmin, lmax = range_select.pop()
print("Order: {}\nFilter: {}\nWav min: {}\nWav max: {}\n".format(order, grism_image.meta.instrument.filter, lmin, lmax))

#### Moving on, let's make boxes for each dispersed spectral object using the corner locations and the wavelengths
SkyObject below is used by the WFSS methods to track grism objects from the catalog, we can display the parts of the catalog it cares about:

In [ ]:
from jwst.lib.catalog_utils import SkyObject
required_fields = list(SkyObject()._fields)
print(required_fields)

#### Get the information we need from the catalog

In [ ]:
from jwst.assign_wcs import util
grism_objects = util.get_object_info(skycoord_cat)

In [ ]:
grism_objects[18]

#### We're going to do this example for just the FIRST SPECTRAL ORDER
Also, remember that `wcs` is the FITS wcs object that we created from the direct image

#### Translated locations to effective grism image "direct image"
    gdet_bbox_ll_x, gdet_bbox_ll_y  = skycoord_to_pixel(catalog['sky_bbox_ll'], grism_fits_wcs, origin=0)
    gdet_bbox_lr_x, gdet_bbox_lr_y  = skycoord_to_pixel(catalog['sky_bbox_lr'], grism_fits_wcs, origin=0)
    gdet_bbox_ul_x, gdet_bbox_ul_y  = skycoord_to_pixel(catalog['sky_bbox_ul'], grism_fits_wcs, origin=0)
    gdet_bbox_ur_x, gdet_bbox_ur_y  = skycoord_to_pixel(catalog['sky_bbox_ur'], grism_fits_wcs, origin=0)

## Replace the NaN values for abmag with something that will pass the code for now

In [ ]:
tempcat=QTable.read(skycoord_cat,  format='ascii.ecsv')
tempcat["abmag"][:] = 18.
tempcat.write(skycoord_cat, overwrite=True)

In [ ]:
tempcat[:][18]

### I've added the `use_fits_wcs` parameter here to make pipeline testing easier until `source_catalog` uses the gwcs object for location translation

In [ ]:
reference_file_names['wavelengthrange'] = 'nircam_wfss_wavelengthrange.asdf'  # use our new file
grism_with_wcs.meta.source_catalog.filename= skycoord_cat  # assign the catalog we made to the image
test_boxes=assign_wcs.util.create_grism_bbox(grism_with_wcs, reference_file_names, use_fits_wcs=True)

### Take a look at one of the brighter objects (catalog #19)


In [ ]:
test_boxes[18]

In [ ]:
test_boxes[-1]

In [ ]:
for source in test_boxes:
    try:
        bounding = source.order_bounding[2]
        ymin,ymax = bounding[0]
        xmin,xmax = bounding[1]
        print("Found: {}\t{}".format(source.sid, source.order_bounding))
    except:
        pass

In [ ]:
### Verify that the centroid of the object got calculated correctly 
x,y = skycoord_to_pixel(catalog[18]['sky_centroid'], grism_fits_wcs, origin=0)
print(x,y)

### Visual verification of order bounding in the grism image

In [ ]:
# visualize the images with some bounding boxes
ys,xs = dirim.shape  # assuming both are the same shape
fig2 = plt.figure(figsize=(8,8), dpi=150)

# Plot the direct image
ax1 = fig2.add_subplot(1, 2, 1)
ax1.ticklabel_format(useOffset=False)
ax1.set_title(directr_file.split("/")[-1], fontsize=8)

# Plot the grism image
ax2 = fig2.add_subplot(1, 2, 2)
ax1.ticklabel_format(useOffset=False)
ax2.set_title(grismr_file.split("/")[-1], fontsize=8)

# rectangle patches are xmin, ymin, width, height
order=1
tt = [test_boxes[18], test_boxes[12], test_boxes[8]]
for obj in tt:
    try:
        y,x  = obj.order_bounding[order]
        ymin, ymax = y
        xmin, xmax = x
        xcenter, ycenter = obj.xcentroid, obj.ycentroid
        ax2.add_patch(patches.Rectangle((xmin, ymin),
                                     xmax - xmin,
                                     ymax - ymin,
                                     fill=False, color='red'))
        ax2.add_patch(patches.Circle((xcenter, ycenter), 10, color='blue'))
    except KeyError:
        pass

ax1.imshow(dirim, origin='lower', extent=[0,xs,0,ys], vmin=-10, vmax=10)
ax2.imshow(dispim, origin='lower', extent=[0,xs,0,ys], vmin=-10, vmax=10)
fig2.tight_layout()
figname = grismr_file.split("/")[-1]+'_order_bounding.jpg'
fig2.savefig(figname, dpi=120, bbox_inches='tight')
print("Saved figure to {}".format(figname))

## The boxes are off a bit? The source location and boxes that were drawn are where the code expects traces to fall for F444W, but the image pixel data looks more like F335M
What if we translate the boxes assuming that the filter *was* F335M

In [ ]:
grismr_335m_file = nircam_data+"V54321001002P0000000001101_A5_F444W_FinalDispersedRamp_row_rate_f335m.fits"

In [ ]:
# Lets read the file into a datamodel and display some information
grism_image = image.ImageModel(grismr_335m_file)
grism_image.meta.instrument.name, grism_image.meta.instrument.filter, grism_image.meta.instrument.pupil, grism_image.meta.instrument.detector,grism_image.meta.exposure.type

In [ ]:
wcs_step = assign_wcs.AssignWcsStep()

In [ ]:
grism_with_wcs = wcs_step(grism_image)

In [ ]:
skycoord_cat='step_SourceCatalogStep_cat.ecsv'  # this should be the name in the output listing above
catalog=QTable.read(skycoord_cat,  format='ascii.ecsv')
catalog

#### the `source_catalog` step returns the xcentroid and ycentroid in pixel values, let's plot those over our image to see how the detections look

In [ ]:
dirim = fits.getdata(directr_file, ext=1)
xs,ys=dirim.shape
fig = plt.figure(figsize=(4,4), dpi=150)
ax = fig.add_subplot(1, 1, 1)
ax.ticklabel_format(useOffset=False)
ax.set_title(directr_file.split("/")[-1], fontsize=8)
ax.imshow(dirim, origin='lower', extent=[0,xs,0,ys], vmin=-10, vmax=10)

# rectangle patches are xmin, ymin, width, height
plist1=[]
for obj in catalog:
    plist1.append(patches.Circle((obj['xcentroid'].value, obj['ycentroid'].value),4, color='red'))
    ax.text(obj['xcentroid'].value, obj['ycentroid'].value, obj['id'], size=5)

for p in plist1:
    ax.add_patch(p)
    
ax.imshow(dirim, origin='lower', extent=[0,xs,0,ys], vmin=-10, vmax=10)
fig.savefig('nircam_wfss_centers_f335m.jpg')

In [ ]:
grism_fits_wcs = grism_with_wcs.get_fits_wcs()
grism_fits_wcs

In [ ]:
# compare with the wcsinfo in the datamodel
grism_with_wcs.meta.wcsinfo.crval1, grism_with_wcs.meta.wcsinfo.crval2, grism_with_wcs.meta.wcsinfo.crpix1, grism_with_wcs.meta.wcsinfo.crpix2

### Assuming that the grism image we are working with has the correct meta information about it's pointing, we can use the RA,DEC coordinates of the corners of the boxes to translate to a respective "direct image" that might have been taken at the same location as the grism image pointing.

In [ ]:
# Translated locations to effective grism image "direct image" using the FITS wcs information
gdet_bbox_ll_x, gdet_bbox_ll_y  = skycoord_to_pixel(catalog['sky_bbox_ll'], grism_fits_wcs, origin=0)
gdet_bbox_lr_x, gdet_bbox_lr_y  = skycoord_to_pixel(catalog['sky_bbox_lr'], grism_fits_wcs, origin=0)
gdet_bbox_ul_x, gdet_bbox_ul_y  = skycoord_to_pixel(catalog['sky_bbox_ul'], grism_fits_wcs, origin=0)
gdet_bbox_ur_x, gdet_bbox_ur_y  = skycoord_to_pixel(catalog['sky_bbox_ur'], grism_fits_wcs, origin=0)
xcenter,ycenter = skycoord_to_pixel(catalog['sky_centroid'], grism_fits_wcs, origin=0)

### Visual display of what the translation from object locations on our direct image to object locations on our grism "direct image"
The boxes in the grism image are at the location of the obect in the catalog, but at the pointing of the grism image so they should show the dither offset
The blue dots on the grism image mark the object center, just as translated by the fits_wcs, not the full trace size. See https://jwst-docs.stsci.edu/display/JTI/NIRCam+Grisms figure 3 for the example of how the source position relates to the trace location.

In [ ]:
# display and save a jpeg copy of the results 
dispim=fits.getdata(grismr_file, ext=1)
dirim=fits.getdata(directr_file, ext=1)
ys,xs=dirim.data.shape
fig2 = plt.figure(figsize=(8,8), dpi=150)
ax1 = fig2.add_subplot(1, 2, 1)
ax1.ticklabel_format(useOffset=False)
ax1.set_title(directr_file.split("/")[-1], fontsize=8)
ax2 = fig2.add_subplot(1, 2, 2)
ax2.set_title(grismr_file.split("/")[-1], fontsize=8)

# rectangle patches are xmin, ymin, width, height
plist1=[]
for xmin,ymin,xmax,ymax,xc,yc in zip(gdet_bbox_ll_x, gdet_bbox_ll_y, gdet_bbox_ur_x, gdet_bbox_ur_y,xcenter,ycenter):
    plist1.append(patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, fill=False, color='red'))
    plist1.append(patches.Circle((xc, yc),4, color='blue'))

for p in plist1:
    ax2.add_patch(p)
    
plist1=[]
for xmin,ymin,xmax,ymax in zip(gdet_bbox_ll_x, gdet_bbox_ll_y, gdet_bbox_ur_x, gdet_bbox_ur_y):
    plist1.append(patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, fill=False, color='red'))
    plist1.append(patches.Circle((xmin, ymin),4, color='blue'))

for p in plist1:
    ax1.add_patch(p)
    
    
ax1.imshow(dirim, origin='lower', extent=[0,xs,0,ys], vmin=-10, vmax=10)
ax2.imshow(dispim, origin='lower', extent=[0,xs,0,ys], vmin=-5, vmax=5)
saveto = grismr_file.split("/")[-1].split(".fits")[0]+"_f335m.jpg"
fig2.tight_layout()
fig2.savefig(saveto, dpi=120, bbox_inches='tight')
print("Saved jpeg file to: {}".format(saveto))

In [ ]:
from jwst import assign_wcs
from jwst.extract_2d.extract_2d_step import Extract2dStep
e2d = Extract2dStep()
ref_types = e2d.reference_file_types
ref_types

reference_file_names = {}
for name in ref_types:
    reffile = e2d.get_reference_file(grism_with_wcs, name)
    reference_file_names[name] = reffile if reffile else ""
reference_file_names

reference_file_names['wavelengthrange'] = 'nircam_wfss_wavelengthrange.asdf'  # use our new file
grism_with_wcs.meta.source_catalog.filename=skycoord_cat  # assign the catalog we made to the image
test_boxes=assign_wcs.util.create_grism_bbox(grism_with_wcs, reference_file_names, use_fits_wcs=True)

### Visual verification of order bounding in the grism image

In [ ]:
# visualize the images with some bounding boxes
ys,xs = dirim.shape  # assuming both are the same shape
fig2 = plt.figure(figsize=(8,8), dpi=150)

# Plot the direct image
ax1 = fig2.add_subplot(1, 2, 1)
ax1.ticklabel_format(useOffset=False)
ax1.set_title(directr_file.split("/")[-1], fontsize=8)

# Plot the grism image
ax2 = fig2.add_subplot(1, 2, 2)
ax1.ticklabel_format(useOffset=False)
ax2.set_title(grismr_file.split("/")[-1], fontsize=8)

# rectangle patches are xmin, ymin, width, height
order=1
tt = [test_boxes[18], test_boxes[12], test_boxes[8]]
for obj in tt:
    try:
        y,x  = obj.order_bounding[order]
        ymin, ymax = y
        xmin, xmax = x
        xcenter, ycenter = obj.xcentroid, obj.ycentroid
        ax2.add_patch(patches.Rectangle((xmin, ymin),
                                     xmax - xmin,
                                     ymax - ymin,
                                     fill=False, color='red'))
        ax2.add_patch(patches.Circle((xcenter, ycenter), 10, color='blue'))
    except KeyError:
        pass

ax1.imshow(dirim, origin='lower', extent=[0,xs,0,ys], vmin=-10, vmax=10)
ax2.imshow(dispim, origin='lower', extent=[0,xs,0,ys], vmin=-10, vmax=10)
fig2.tight_layout()
figname = grismr_file.split("/")[-1]+'_order_bounding_f335m.jpg'
fig2.savefig(figname, dpi=120, bbox_inches='tight')
print("Saved figure to {}".format(figname))

    [1, 'F335M', 3.01459734, 4.260432726]
    [1, 'F444W', 3.696969216, 4.899565197]
    

### If I continue on, let's just make sure that the order is extracted in the correct location for F335M

In [ ]:
from jwst.extract_2d import extract_2d_step, extract_2d
step=extract_2d_step.Extract2dStep()
reference_file_names = {'camera': 'N/A',
 'collimator': 'N/A',
 'disperser': 'N/A',
 'distortion': step.get_reference_file(grism_with_wcs,'distortion'),
 'filteroffset': 'N/A',
 'fore': 'N/A',
 'fpa': 'N/A',
 'ifufore': 'N/A',
 'ifupost': 'N/A',
 'ifuslicer': 'N/A',
 'msa': 'N/A',
 'ote': 'N/A',
 'regions': 'N/A',
 'specwcs':  step.get_reference_file(grism_with_wcs,'specwcs'),
 'v2v3': 'N/A',
 'wavelengthrange': step.get_reference_file(grism_with_wcs, 'wavelengthrange')}

In [ ]:
reference_file_names['wavelengthrange'] = 'nircam_wfss_wavelengthrange.asdf'  # use our new file

In [ ]:
x2d = extract_2d.extract_grism_objects(grism_with_wcs, reference_files=reference_file_names, mmag_extract=99., use_fits_wcs=True)

In [ ]:
len(x2d.slits)

In [ ]:
x2d.write('nircam_extract_2d_wfss_slits_f335M.fits')
!ls -altr *.fits

In [ ]:
dispim=fits.getdata(grismr_file)
dirim=fits.getdata(directr_file)
ys,xs=dirim.data.shape

fig = plt.figure(figsize=(10,10), dpi=150)
ax3 = fig.add_subplot(1, 2, 1)
ax3.set_title(directr_file.split("/")[-1], fontsize=8)
ax3.imshow(dirim, origin='lower', extent=[0,xs,0,ys], vmin=-3, vmax=3)
ax2 = fig.add_subplot(1, 2, 2)
ax2.set_title(grismr_file.split("/")[-1], fontsize=8)
ax2.imshow(dispim, origin='lower', extent=[0,xs,0,ys], vmin=-3, vmax=3)
fig.tight_layout()


# rectangle patches are xmin, ymin, width, height
plist1=[]
for obj in catalog:
    plist1.append(patches.Circle((obj['xcentroid'].value, obj['ycentroid'].value),4, color='red'))
    ax3.text(obj['xcentroid'].value, obj['ycentroid'].value, obj['id'], size=5)

for p in plist1:
    ax3.add_patch(p)
ax3.imshow(dirim, origin='lower', extent=[0,xs,0,ys], vmin=-3, vmax=3)

In [ ]:
fig = plt.figure(figsize=(8,6), dpi=150)

ss=30
ys, xs = x2d.slits[ss].data.shape
ax = fig.add_subplot(3, 1, 1)
#xpos, ypos = x2d.slits[ss].source_xpos, x2d.slits[ss].source_ypos
#title = x2d.meta.instrument.filter+" order {0}\nx={1} y={2}".format(x2d.meta.wcsinfo.spectral_order,
#                                                                    xpos,
#                                                                    ypos)
title = "object {} order = {}".format(x2d.slits[ss].source_id, x2d.slits[ss].meta.wcsinfo.spectral_order)
ax.set_title(title, fontsize=8)
ax.imshow(x2d.slits[ss].data, origin='lower', extent=[0,xs,0,ys], vmin=-3, vmax=3)

ss=14
ys, xs = x2d.slits[ss].data.shape
ax4 = fig.add_subplot(3, 1, 2)
#xpos, ypos = x2d.slits[ss].source_xpos, x2d.slits[ss].source_ypos
#title = x2d.meta.instrument.filter+" order {0}\nx={1} y={2}".format(x2d.meta.wcsinfo.spectral_order,
#                                                                    xpos,
#                                                                    ypos)
title = "object {} order = {}".format(x2d.slits[ss].source_id, x2d.slits[ss].meta.wcsinfo.spectral_order)
ax4.set_title(title, fontsize=8)
ax4.imshow(x2d.slits[ss].data, origin='lower', extent=[0,xs,0,ys], vmin=-3, vmax=3)
figname = grismr_file.split("/")[-1]+'_order_extracted_f335m.jpg'
print("Saved image to {}".format(figname))
fig2.savefig(figname, dpi=120, bbox_inches='tight')

In [ ]:
x2d.slits[30].source_id

In [ ]:
x2d.slits[30].meta.wcs.bounding_box

In [ ]:
x2d.slits[30].source_xpos, x2d.slits[30].source_ypos #in full frame coordinates

In [ ]:
x2d.meta.wcsinfo.crval1, x2d.meta.wcsinfo.crval2

In [ ]:
x2d.slits[30].meta.wcs(300, 45)

In [ ]:
x2d.slits[30].meta.wcs.invert(53.24559353894091, -27.67442607541429, 3.6600624374534227, 1.0)  


In [ ]:
x2d.slits[30].meta.wcs.available_frames